In [4]:
import json
import pandas as pd
import nltk
from nltk.tokenize import TweetTokenizer as tw_tokenizer
from unidecode import unidecode
import time
import math

kelompok2_1 = pd.read_csv('data/kelompok2_1.csv')
dataset=[]
for tweet in kelompok2_1['text']:
    dataset.append(tweet)

In [5]:
tweetsIn = pd.DataFrame({'text':dataset})
tweetsIn.head()

,text
0,@sadbness Klo bulanan kan happy menstruasi
1,"Jalan ke warteg aja mas deket, dah gt murah, k..."
2,Siapin lilin dulu dah
3,#DebatCapresJICT \r\n#asingkuasaipelabuhanJICT
4,"quote quote \r\n\r\n"" hasil tidak akan mengkhi..."


In [11]:
stopword_file1 = open('stopword/stopword_id.txt', "r").read() # Membuka file stopword bahasa indonesia dan menjadikan isi file tersebut sebagai string
stopword_file2 = open('stopword_en/stopwords_en.txt', "r").read()  # Membuka file stopword bahasa inggris dan menjadikan isi file tersebut sebagai string
stopword_file3 = open('stopword_noise/stopword_noise.txt', "r").read()  # Membuka file stopword noise dan menjadikan isi file tersebut sebagai string
stopword_file_all = stopword_file1 + stopword_file2 + stopword_file3  # Menggabungkan ketiga string stopword sebelumnya kedalam satu string
stopwords = stopword_file_all.split('\n') # Memisahkan kata dalam string yang sudah digambungkan berdasarkan baris
# print(stopwords)

In [18]:
slangwords = dict() # Membuat dictionary kosong untuk menyimpan kata slang dan formal sebagai key dan value
slangwords_dataframe = pd.read_csv('slangword/colloquial-indonesian-lexicon.csv') # Membuka file csv yang berisi kata slang dan formal dan mengkonversi kedalam dataframe
for slang, formal in zip(slangwords_dataframe['slang'], slangwords_dataframe['formal']):
    slangwords[slang] = formal # Mapping kata slang dan formal dan memasukkan ke dalam dictionary secara berulang

slangword_file = open('slangword/slangword.txt', "r").read() # Membuka file yang berisi kata slang dan kata formal dan mengkonversi kedalam string
slangwords_text = slangword_file.split('\n') # Memisahkan kata berdasarkan baris namun kata slang dan kata formal masih belum terpisah. output : (['slang:formal', ...])
#print(slangwords_text)
for slang in slangwords_text:
    split_slang = slang.split(":") # Memisahkan semua kata slang dan kata formal berdasarkan "titik dua (:)"
    slangwords[split_slang[0]] = split_slang[1] # Mapping semua kata slang dan kata formal ke dalam dictionary. Output : {'slang' : 'formal', ...}
#print(slangwords)

In [19]:
tweetsIn['tokenized'] = list(map(lambda tweet: tw_tokenizer(strip_handles=True, reduce_len=True).tokenize(unidecode(tweet).lower()), tweetsIn['text'])) # Memisahkan kata dalam text berasarkan "spasi"
tweetsIn['clean_text'] = list(map(lambda tweet: [w for w in tweet if w.isalnum()], tweetsIn['tokenized'])) # Filtering kata yang hanya berisi karakater a-z dan 0-9 (Menghapus url, hashtag, mention)
tweetsIn.head()

,text,tokenized,clean_text
0,@sadbness Klo bulanan kan happy menstruasi,"[klo, bulanan, kan, happy, menstruasi]","[klo, bulanan, kan, happy, menstruasi]"
1,"Jalan ke warteg aja mas deket, dah gt murah, k...","[jalan, ke, warteg, aja, mas, deket, ,, dah, g...","[jalan, ke, warteg, aja, mas, deket, dah, gt, ..."
2,Siapin lilin dulu dah,"[siapin, lilin, dulu, dah]","[siapin, lilin, dulu, dah]"
3,#DebatCapresJICT \r\n#asingkuasaipelabuhanJICT,"[#debatcapresjict, #asingkuasaipelabuhanjict]",[]
4,"quote quote \r\n\r\n"" hasil tidak akan mengkhi...","[quote, quote, "", hasil, tidak, akan, mengkhia...","[quote, quote, hasil, tidak, akan, mengkhianat..."


In [20]:
tweetsIn['handled_slangword'] = list(map(lambda tweet : list(map(lambda w : slangwords[w] if w in slangwords.keys() else w, tweet)), tweetsIn['clean_text']))
# Mengubah kata slang menjadi kata formal (kata slang dan kata formal yang diperoleh dari dictionary) 
#print(handled_slangword[:100])
tweetsIn.head(10)

,text,tokenized,clean_text,handled_slangword
0,@sadbness Klo bulanan kan happy menstruasi,"[klo, bulanan, kan, happy, menstruasi]","[klo, bulanan, kan, happy, menstruasi]","[kalau, bulanan, kan, happy, menstruasi]"
1,"Jalan ke warteg aja mas deket, dah gt murah, k...","[jalan, ke, warteg, aja, mas, deket, ,, dah, g...","[jalan, ke, warteg, aja, mas, deket, dah, gt, ...","[jalan, ke, warteg, saja, mas, dekat, deh, beg..."
2,Siapin lilin dulu dah,"[siapin, lilin, dulu, dah]","[siapin, lilin, dulu, dah]","[siapin, lilin, dahulu, deh]"
3,#DebatCapresJICT \r\n#asingkuasaipelabuhanJICT,"[#debatcapresjict, #asingkuasaipelabuhanjict]",[],[]
4,"quote quote \r\n\r\n"" hasil tidak akan mengkhi...","[quote, quote, "", hasil, tidak, akan, mengkhia...","[quote, quote, hasil, tidak, akan, mengkhianat...","[quote, quote, hasil, tidak, akan, mengkhianat..."
5,[23:56] #JAKARTA Tol Tomang - Karang Tengah - ...,"[[, 23:56, ], #jakarta, tol, tomang, -, karang...","[tol, tomang, karang, tengah, tangerang, bitung]","[tol, tomang, karang, tengah, tangerang, bitung]"
6,[23:56] #JAKARTA Tol Cengkareng - Pluit - Toma...,"[[, 23:56, ], #jakarta, tol, cengkareng, -, pl...","[tol, cengkareng, pluit, tomang, cawang]","[tol, cengkareng, pluit, tomang, cawang]"
7,[23:56] #JAKARTA Tol Cawang - Pancoran #LANCAR...,"[[, 23:56, ], #jakarta, tol, cawang, -, pancor...","[tol, cawang, pancoran]","[tol, cawang, pancoran]"
8,[23:56] #JAKARTA Buaran - Cipinang #LANCAR #Ja...,"[[, 23:56, ], #jakarta, buaran, -, cipinang, #...","[buaran, cipinang]","[buaran, cipinang]"
9,[23:55] #JAKARTA Tol Semper - Cakung - Cikunir...,"[[, 23:55, ], #jakarta, tol, semper, -, cakung...","[tol, semper, cakung, cikunir, tmii]","[tol, semper, cakung, cikunir, tmii]"


In [21]:
tweetsIn['removed_stopwords'] = list(map(lambda tweet : [w for w in tweet if w not in stopwords], tweetsIn['handled_slangword']))
# Filtering data dengan menghapus kata yang tidak bermakna (Stopword yang diperoleh dari file)
tweetsIn.head(10)

,text,tokenized,clean_text,handled_slangword,removed_stopwords
0,@sadbness Klo bulanan kan happy menstruasi,"[klo, bulanan, kan, happy, menstruasi]","[klo, bulanan, kan, happy, menstruasi]","[kalau, bulanan, kan, happy, menstruasi]","[bulanan, happy, menstruasi]"
1,"Jalan ke warteg aja mas deket, dah gt murah, k...","[jalan, ke, warteg, aja, mas, deket, ,, dah, g...","[jalan, ke, warteg, aja, mas, deket, dah, gt, ...","[jalan, ke, warteg, saja, mas, dekat, deh, beg...","[jalan, warteg, murah, tidak punya, uang, engg..."
2,Siapin lilin dulu dah,"[siapin, lilin, dulu, dah]","[siapin, lilin, dulu, dah]","[siapin, lilin, dahulu, deh]","[siapin, lilin]"
3,#DebatCapresJICT \r\n#asingkuasaipelabuhanJICT,"[#debatcapresjict, #asingkuasaipelabuhanjict]",[],[],[]
4,"quote quote \r\n\r\n"" hasil tidak akan mengkhi...","[quote, quote, "", hasil, tidak, akan, mengkhia...","[quote, quote, hasil, tidak, akan, mengkhianat...","[quote, quote, hasil, tidak, akan, mengkhianat...","[quote, quote, hasil, mengkhianati, proses, ny..."
5,[23:56] #JAKARTA Tol Tomang - Karang Tengah - ...,"[[, 23:56, ], #jakarta, tol, tomang, -, karang...","[tol, tomang, karang, tengah, tangerang, bitung]","[tol, tomang, karang, tengah, tangerang, bitung]","[tol, tomang, karang, tangerang, bitung]"
6,[23:56] #JAKARTA Tol Cengkareng - Pluit - Toma...,"[[, 23:56, ], #jakarta, tol, cengkareng, -, pl...","[tol, cengkareng, pluit, tomang, cawang]","[tol, cengkareng, pluit, tomang, cawang]","[tol, cengkareng, pluit, tomang, cawang]"
7,[23:56] #JAKARTA Tol Cawang - Pancoran #LANCAR...,"[[, 23:56, ], #jakarta, tol, cawang, -, pancor...","[tol, cawang, pancoran]","[tol, cawang, pancoran]","[tol, cawang, pancoran]"
8,[23:56] #JAKARTA Buaran - Cipinang #LANCAR #Ja...,"[[, 23:56, ], #jakarta, buaran, -, cipinang, #...","[buaran, cipinang]","[buaran, cipinang]",[buaran]
9,[23:55] #JAKARTA Tol Semper - Cakung - Cikunir...,"[[, 23:55, ], #jakarta, tol, semper, -, cakung...","[tol, semper, cakung, cikunir, tmii]","[tol, semper, cakung, cikunir, tmii]","[tol, semper, cakung, cikunir, tmii]"


In [22]:
from spacy.lang.id import Indonesian # Import modul spacy bahasa indonesia

nlp = Indonesian() # memanggi objek Indonesian() pada modul spacy
def stem_spacy(text): # Fungsi untuk mengubah kata-kata menjadi kata dasar
    for txt in nlp(text):
        t = txt.lemma_
    return t
tweetsIn['stemmed_by_spacy'] = list(map(lambda tweet : list(map(lambda word: stem_spacy(word), tweet)), tweetsIn['removed_stopwords']))
#tweetsIn.to_csv("data/" + str(time.time()) + "_export_clean_text.csv")
tweetsIn.head(10)

,text,tokenized,clean_text,handled_slangword,removed_stopwords,stemmed_by_spacy
0,@sadbness Klo bulanan kan happy menstruasi,"[klo, bulanan, kan, happy, menstruasi]","[klo, bulanan, kan, happy, menstruasi]","[kalau, bulanan, kan, happy, menstruasi]","[bulanan, happy, menstruasi]","[bulan, happy, menstruasi]"
1,"Jalan ke warteg aja mas deket, dah gt murah, k...","[jalan, ke, warteg, aja, mas, deket, ,, dah, g...","[jalan, ke, warteg, aja, mas, deket, dah, gt, ...","[jalan, ke, warteg, saja, mas, dekat, deh, beg...","[jalan, warteg, murah, tidak punya, uang, engg...","[jalan, warteg, murah, punya, uang, usah, soka..."
2,Siapin lilin dulu dah,"[siapin, lilin, dulu, dah]","[siapin, lilin, dulu, dah]","[siapin, lilin, dahulu, deh]","[siapin, lilin]","[siapin, lilin]"
3,#DebatCapresJICT \r\n#asingkuasaipelabuhanJICT,"[#debatcapresjict, #asingkuasaipelabuhanjict]",[],[],[],[]
4,"quote quote \r\n\r\n"" hasil tidak akan mengkhi...","[quote, quote, "", hasil, tidak, akan, mengkhia...","[quote, quote, hasil, tidak, akan, mengkhianat...","[quote, quote, hasil, tidak, akan, mengkhianat...","[quote, quote, hasil, mengkhianati, proses, ny...","[quote, quote, hasil, khianat, proses, nyata]"
5,[23:56] #JAKARTA Tol Tomang - Karang Tengah - ...,"[[, 23:56, ], #jakarta, tol, tomang, -, karang...","[tol, tomang, karang, tengah, tangerang, bitung]","[tol, tomang, karang, tengah, tangerang, bitung]","[tol, tomang, karang, tangerang, bitung]","[tol, tomang, karang, tangerang, bitung]"
6,[23:56] #JAKARTA Tol Cengkareng - Pluit - Toma...,"[[, 23:56, ], #jakarta, tol, cengkareng, -, pl...","[tol, cengkareng, pluit, tomang, cawang]","[tol, cengkareng, pluit, tomang, cawang]","[tol, cengkareng, pluit, tomang, cawang]","[tol, cengkareng, pluit, tomang, cawang]"
7,[23:56] #JAKARTA Tol Cawang - Pancoran #LANCAR...,"[[, 23:56, ], #jakarta, tol, cawang, -, pancor...","[tol, cawang, pancoran]","[tol, cawang, pancoran]","[tol, cawang, pancoran]","[tol, cawang, pancoran]"
8,[23:56] #JAKARTA Buaran - Cipinang #LANCAR #Ja...,"[[, 23:56, ], #jakarta, buaran, -, cipinang, #...","[buaran, cipinang]","[buaran, cipinang]",[buaran],[buaran]
9,[23:55] #JAKARTA Tol Semper - Cakung - Cikunir...,"[[, 23:55, ], #jakarta, tol, semper, -, cakung...","[tol, semper, cakung, cikunir, tmii]","[tol, semper, cakung, cikunir, tmii]","[tol, semper, cakung, cikunir, tmii]","[tol, semper, cakung, cikunir, tmii]"


In [23]:
tweetsInreset = tweetsIn.reset_index()
DATA = tweetsInreset['stemmed_by_spacy']
N = 5 # Banyaknya

In [25]:
def create_term(text_list, total_text = 0):
    wordcloud = []
    if total_text == 0:
        total_text = len(text_list)

    for word in text_list[:total_text]:
        wordcloud += word

    term = list(dict.fromkeys(wordcloud))
    return term

print(create_term(DATA))

['bulan', 'happy', 'menstruasi', 'jalan', 'warteg', 'murah', 'punya', 'uang', 'usah', 'sokan', 'susah', 'bangsat', 'siapin', 'lilin', 'quote', 'hasil', 'khianat', 'proses', 'nyata', 'tol', 'tomang', 'karang', 'tangerang', 'bitung', 'cengkareng', 'pluit', 'cawang', 'pancoran', 'buaran', 'semper', 'cakung', 'cikunir', 'tmii', 'serpong', 'ranji', 'ulujami', 'tunggu', 'emaknya', 'pili', 'ati', 'harganya', 'yaudah', 'dipermasalahin', 'bahagian', 'ih', 'ribet', 'mudah', 'pokoknya', 'dahhh', 'desir', 'pasir', 'padang', 'tandus', 'ketemu', 'kian', 'berkah', 'sahabat', 'kasih', 'allah', 'adu', 'diwaktu', 'dicari', 'dipikir', 'sesat', 'mending', 'alesan', 'tanggal', 'segitu', 'diajak', 'keluargamu', 'libur', 'urus', 'kuliah', 'keterima', 'ker', 'kangen', 'bucin', 'ku', 'momen', 'keluarnya', 'mate', 'series', 'faktor', 'p30', 'jgd', 'hebring', 'nyaingin', 'intinya', 'pro', 'yek', 'trans', 'park', 'debat', 'kawan', 'cari', 'perhati', 'bicara', 'fantasi', 'keren', 'duduk', 'tolong', 'life', 'circum

In [26]:
def create_tf(text_list, total_text= 0):
    if total_text == 0:
        total_text = len(text_list)
    term = create_term(text_list, total_text)
    tf = dict()
    for i in range(total_text):
        name = 'tf_text_' + str(i+1)
        tf[name] = list(map(lambda word: text_list[i].count(word), term))
    return tf

tf = create_tf(DATA)
# tf = create_tf(DATA, N)

pd.DataFrame(tf)

MemoryError: 

In [ ]:
def create_idf(text_list, total_text = 0):
    if total_text == 0:
        total_text = len(text_list)
    idf = []
    term = create_term(text_list, total_text)
    tf = create_tf(text_list, total_text)
    total_words = len(term)

    for i in range(total_words):
        val = 0
        for j in range(total_text):
            name = 'tf_text_' + str(j+1)
            if float(tf[name][i]) > 0:
                val += 1
        idf.append(math.log10(total_text / float(val)))
    return idf

idf = create_idf(DATA)
# tf = create_idf(DATA, N)
idf